In [1]:
import numpy as np

In [2]:
f = open('glove.6B.200d.txt','r')
dictionary = {}
vectors = []
for no, line in enumerate(f):
    splitLine = line.split()
    word = splitLine[0]
    dictionary[word] = no
    embedding = np.array([float(val) for val in splitLine[1:]])
    vectors.append(embedding)

In [5]:
vectors = np.array(vectors)

In [24]:
rev_dictionary = {v:k for k, v in dictionary.items()}

In [7]:
import tensorflow as tf

In [10]:
class Model:
    def __init__(self):
        self._embedding = tf.convert_to_tensor(vectors, dtype = tf.float32)
        self.X = tf.placeholder(
                tf.float32, [None, vectors.shape[1]]
        )
        normed_embedding = tf.nn.l2_normalize(self._embedding, axis = 1)
        normed_array = tf.nn.l2_normalize(self.X, axis = 1)
        self.cosine_similarity = tf.matmul(
            normed_array, tf.transpose(normed_embedding, [1, 0])
        )

In [11]:
model = Model()
sess = tf.InteractiveSession()

In [14]:
string_positive = 'i really love to eat chicken and meat'
string_negative = 'i really hate you and i do not want to see you again'

In [28]:
import random

In [52]:
def augmentation(string, threshold = 0.5, count = 5, k = 8):
    string = string.split()
    selected = []
    while not len(selected):
        selected = [(no, w) for no, w in enumerate(string) if random.random() > threshold]
    indices, words = [i[0] for i in selected], [i[1] for i in selected]
    
    batches = vectors[[dictionary[w] for w in words]]
    top_k = tf.nn.top_k(model.cosine_similarity, k = k)
    results = sess.run(top_k, feed_dict = {model.X: batches})
    words = []
    for result in results.indices:
        words.append([rev_dictionary[i] for i in result])
    augmented = []
    for i in range(count):
        string_ = string[:]
        for no in range(len(words)):
            index = random.randint(0, len(words[no]) - 1)
            string_[indices[no]] = words[no][index]
        augmented.append(' '.join(string_))
    return augmented   

In [53]:
%%time
augmentation(string_positive)

CPU times: user 4.07 s, sys: 1.59 s, total: 5.66 s
Wall time: 5.68 s


['i thing love to eat pork also chicken',
 'i really love to eat fried well cooked',
 'i things love to eat meat with beef',
 'i thing love to eat roasted well chicken',
 'i something love to eat cooked , beef']

In [54]:
%%time
augmentation(string_negative)

CPU times: user 3.96 s, sys: 1.78 s, total: 5.73 s
Wall time: 5.75 s


['i really hate you also i know not want make see i again',
 "i really hate you while i n't not want help see 'll again",
 "i really hate you . i not not want take see 'll again",
 'i really hate you well i know not want able see you again',
 "i really hate you , i does not want could see 'll again"]